# 1.Introducción


# 2.Armado del ambiente


In [ ]:
#--------------------------------------------------------------
#@title 2.1 Parametros de ejecución { vertical-output: true }
#@markdown ---
#@markdown ### Instalacion de bibliotecas de Python


# 3.Desarrollo


In [2]:
#--------------------------------------------------------------
#@title 3.1 Parametros de ejecución { vertical-output: true }

cantidad_N =  10#@param {type: "number"}

from sympy import Derivative, diff, simplify

fx = (2*x + 1)*(x**3 + 2)
dx = Derivative(fx, x).doit()
dx


  

NameError: ignored

#4.Tabla de pasos


# 5.Conclusiones

# 6.Bibliografía
